In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
from skimage import io, transform, exposure
from skimage.transform import ProjectiveTransform, rotate, warp

In [2]:
def save_data(datafile, data):
    with open(datafile, 'wb') as file:
        pickle.dump(data, file)

def load_data(datafile):
    with open(datafile, 'rb') as file:
        return pickle.load(file)

In [3]:
#Loading Training Data
base_dir = os.path.join(os.getcwd(), 'Training_Images')
train_data = {i : [] for i in range(len(os.listdir(base_dir)))}
for i, folder in zip(train_data, os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder)
    for file in os.listdir(folder_path):
        if file.endswith('ppm'):
            file_path = os.path.join(folder_path, file)
            image = np.array(io.imread(file_path))
            image = transform.resize(image, (32, 32), mode='edge')
            train_data[i].append(image)

In [4]:
#Load Test Data
df =  pd.read_csv('Test.csv')
test_data = {
    'feature' : [],
    'label' : []
}

base_dir = os.path.join(os.getcwd(), 'Test_Images')
for i, images in enumerate(os.listdir(base_dir)):
    if images.endswith('ppm'):
        images = os.path.join(base_dir, images)
        images = np.array(io.imread(images))
        images = transform.resize(images, (32, 32), mode ='edge')
        test_data['feature'].append(images)
        test_data['label'].append(df['ClassId'][i])

In [5]:
#Data Functions
def image_flip(feature, label):
    vert = [1, 5, 12, 15, 17]
    hori = [11, 12, 13, 15, 17, 18, 22, 26, 30, 35]
    both = [32, 40]
    chng = {19: 20, 20: 19,
            33: 34, 34: 33,
            36: 37, 37: 36,
            38: 39, 39: 38}

    if label in vert:
        vert_flipped = np.flip(feature, axis=0)
        yield vert_flipped, label
    
    if label in hori:
        hori_flipped = np.flip(feature, axis=1)
        yield hori_flipped, label
    
    if label in both:
        new_image = np.flip(feature, axis=(0, 1))
        yield new_image, label
    
    if label in chng:
        flipped_image = np.flip(feature, axis=1)
        yield flipped_image, chng[label]
    

def image_warp(feature, label=None, intensity=0.75):
    delta = 32 * .3 * intensity
    tl_top = random.uniform(-delta, delta)
    tl_left = random.uniform(-delta, delta)
    bl_bot = random.uniform(-delta, delta)
    bl_left = random.uniform(-delta, delta)
    tr_top = random.uniform(-delta, delta)
    tr_right = random.uniform(-delta, delta)
    br_bot = random.uniform(-delta, delta)
    br_right = random.uniform(-delta, delta)

    tf = ProjectiveTransform()
    tf.estimate(np.array(((tl_left, tl_top), 
                          (bl_left, 32 - bl_bot), 
                          (32 - br_right, 32 - br_bot), 
                          (32 - tr_right, tr_top))),
                np.array(((0, 0),
                          (0, 32),
                          (32, 32),
                          (32, 0)))
                )

    new_image = transform.warp(feature, tf, order=1, mode='edge')
    return new_image, label

def image_rotate(feature, label=None, intensity=0.75):
    delta = 30 * intensity
    new_image = transform.rotate(feature, angle=(random.uniform(-delta, delta)), mode='edge')
    return new_image, label

def grayscale_contrast(feature):
    new_image = 0.2989 * feature[:, :, 0] + 0.5870 * feature[:, :, 1] + 0.1140 * feature[:, :, 2]
    new_image = exposure.equalize_adapthist(new_image)
    return new_image



In [6]:
#Flip Images
dump = [[] for _ in range(len(train_data))]

for i, feature in train_data.items():
    for image in feature:
        for feat, labl in image_flip(image, i):
            dump[labl].append(feat)

for i in range(len(dump)):
    train_data[i] += dump[i]

print("Done Image Flip")

Done Image Flip


In [7]:
#Warp Images
temp = {i : value[:] for i, value in train_data.items()}
max_sample = max([len(temp[i]) for i in range(len(temp))])

for label, feature in train_data.items():
    while len(feature) < max_sample:
        aug_image, lbl = image_rotate(feature[random.randint(0, len(temp[label]) - 1)], label)
        aug_image, lbl = image_warp(aug_image, lbl)
        train_data[lbl].append(aug_image)

print("Done Image Augmentation")

Done Image Augmentation


In [8]:
#Store Data into single list
final_data = {
    'feature': [],
    'label': []
}

for lbl, fet in train_data.items():
    for f in fet:
        final_data['feature'].append(grayscale_contrast(f))
        final_data['label'].append(lbl)

In [9]:
save_data('training_data.pkl', final_data)

In [25]:
train_data = load_data('training_data.pkl')

# Model Design

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import DataLoader, Dataset
from torchvision import io, transforms

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
df = pd.read_csv('trafficsignclassifier.csv')

In [15]:
#Initial Hyperparameters

num_epochs = 50
batch_size = 100
learning_rate = 0.001
classes = tuple([i for i in df['signNames']])


In [16]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[1])
])

In [19]:
class TrafficCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv32 = nn.Conv2d(1, 32, 5, padding=2) 
        self.conv16 = nn.Conv2d(32, 64, 5, padding=2) 
        self.conv4 = nn.Conv2d(64, 128, 5, padding=2)
        self.maxp = nn.MaxPool2d(2, 2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 43)

    def forward(self, x):
        # Conv -> ReLU -> MaxPool -> Conv -> ReLU -> MaxPool -> Conv -> ReLU -> Maxpool
        x = F.relu(self.conv32(x))
        x = self.maxp(x)
        x = F.relu(self.conv16(x))
        x = self.maxp(x)
        x = F.relu(self.conv4(x))
        x = self.maxp(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

class CustomData(Dataset):
    def __init__(self, feature, label, transform=None):
        self.feature = feature
        self.label = label
        self.transform = transform

    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
        sample = self.feature[index]
        label = self.label[index]

        if self.transform:
            sample = self.transform(sample)
        
        return sample, label
    

In [26]:
train_data = CustomData(train_data['feature'], train_data['label'], transform=data_transform)
train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
model = TrafficCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = opt.SGD(model.parameters(), lr=learning_rate)

In [ ]:
total_steps = len(data['feature'])
